#### Получение исторических данных через информационно-статистический сервер Московской биржи IIS
Сервер постоен на архитектуре RESTful API. Запросы будем формировать в виде URL с параметрами.
<br>Информация о сервере доступна здесь: https://www.moex.com/a2193

In [1]:
# импортируем необходимые модули
import pandas as pd
import numpy as np
import os
import requests

#### Получение списка рынков
<br>Запрос https://iss.moex.com/iss/engines.json вернет список рынков в формате json
<br>Структуру json запроса можно визуально посмотреть используя сервис https://jsonformatter.org/

In [2]:
# получим список рынков engines
url_engines='https://iss.moex.com/iss/engines.json'
response_engines=requests.get(url_engines)
if response_engines.status_code!=200:
    raise ValueError(f"Request get engines failed {response_engines.status_code}")
dict_engines=response_engines.json()
print(dict_engines)

{'engines': {'metadata': {'id': {'type': 'int32'}, 'name': {'type': 'string', 'bytes': 45, 'max_size': 0}, 'title': {'type': 'string', 'bytes': 765, 'max_size': 0}}, 'columns': ['id', 'name', 'title'], 'data': [[1, 'stock', 'Фондовый рынок и рынок депозитов'], [2, 'state', 'Рынок ГЦБ (размещение)'], [3, 'currency', 'Валютный рынок'], [4, 'futures', 'Срочный рынок'], [5, 'commodity', 'Товарный рынок'], [6, 'interventions', 'Товарные интервенции'], [7, 'offboard', 'ОТС-система'], [9, 'agro', 'Агро'], [1012, 'otc', 'OTC Система']]}}


In [3]:
# преобразуем json в dataframe
df_engines=pd.DataFrame(dict_engines['engines']['data'],columns=dict_engines['engines']['columns'])
print(df_engines)

     id           name                             title
0     1          stock  Фондовый рынок и рынок депозитов
1     2          state            Рынок ГЦБ (размещение)
2     3       currency                    Валютный рынок
3     4        futures                     Срочный рынок
4     5      commodity                    Товарный рынок
5     6  interventions              Товарные интервенции
6     7       offboard                       ОТС-система
7     9           agro                              Агро
8  1012            otc                       OTC Система


#### Получим список торгуемых инструментов на Срочном рынке futures в классе forts
Запрос https://iss.moex.com/iss/engines/futures/markets вернет список классов Срочного рынка
<br>Нас интересует класс forts
<br>Далее узнаем boardid класса forts и после этого получим список торгуемых инструментов этого класса

In [4]:
# первый способ получения boardid через url запрос csv
url_boardid='https://iss.moex.com/iss/engines/futures/markets/forts/boards.csv'
boardid=pd.read_csv(url_boardid, sep=';',nrows=1,skiprows=1, encoding='windows-1251').boardid.iloc[0]

In [5]:
# второй способ получения boardid через url запрос json
url_boardid='https://iss.moex.com/iss/engines/futures/markets/forts/boards.json'
response_boards=requests.get(url_boardid)
if response_boards.status_code!=200:
    raise ValueError(f"Request get boardid failed {response_boards.status_code}")
dict_boards=response_boards.json()
print(dict_boards)

{'boards': {'metadata': {'id': {'type': 'int32'}, 'board_group_id': {'type': 'int32'}, 'boardid': {'type': 'string', 'bytes': 12, 'max_size': 0}, 'title': {'type': 'string', 'bytes': 381, 'max_size': 0}, 'is_traded': {'type': 'int32'}}, 'columns': ['id', 'board_group_id', 'boardid', 'title', 'is_traded'], 'data': [[101, 45, 'RFUD', 'Фьючерсы', 1]]}}


In [6]:
# рассмотрев структуру dict_boards извлечем boradid
boardid=dict_boards['boards']['data'][0][2]
print('boardid forts: ' + boardid)

boardid forts: RFUD


In [7]:
# Формируем запрос списка инструментов Срочного рынка futures класса forts
url_forts_sec='https://iss.moex.com/iss/engines/futures/markets/forts/boards/'+boardid+'/securities.json'
response_forts_sec=requests.get(url_forts_sec)
if response_forts_sec.status_code!=200:
    raise ValueError(f"Request get forts securities failed {response_forts_sec.status_code}")
dict_forts_sec=requests.get(url_forts_sec).json()


In [8]:
# создадим dataframe
df_forts_sec=pd.DataFrame(dict_forts_sec['securities']['data'],columns=dict_forts_sec['securities']['columns'])
print(df_forts_sec.columns)
print(df_forts_sec[['SECID','BOARDID','SHORTNAME','SECNAME']].head())

Index(['SECID', 'BOARDID', 'SHORTNAME', 'SECNAME', 'PREVSETTLEPRICE',
       'DECIMALS', 'MINSTEP', 'LASTTRADEDATE', 'LASTDELDATE', 'SECTYPE',
       'LATNAME', 'ASSETCODE', 'PREVOPENPOSITION', 'LOTVOLUME',
       'INITIALMARGIN', 'HIGHLIMIT', 'LOWLIMIT', 'STEPPRICE',
       'LASTSETTLEPRICE', 'PREVPRICE', 'IMTIME', 'BUYSELLFEE', 'SCALPERFEE',
       'NEGOTIATEDFEE', 'EXERCISEFEE'],
      dtype='object')
  SECID BOARDID   SHORTNAME                         SECNAME
0  AFM2    RFUD   AFLT-6.22   Фьючерсный контракт AFLT-6.22
1  AFU2    RFUD   AFLT-9.22   Фьючерсный контракт AFLT-9.22
2  AFZ2    RFUD  AFLT-12.22  Фьючерсный контракт AFLT-12.22
3  AKM2    RFUD   AFKS-6.22   Фьючерсный контракт AFKS-6.22
4  AKU2    RFUD   AFKS-9.22   Фьючерсный контракт AFKS-9.22


In [9]:
# выведем список инструменов содержащих символы SR
df_forts_sec[df_forts_sec['SECID'].str.contains('SR',regex=False)]

,SECID,BOARDID,SHORTNAME,SECNAME,PREVSETTLEPRICE,DECIMALS,MINSTEP,LASTTRADEDATE,LASTDELDATE,SECTYPE,...,HIGHLIMIT,LOWLIMIT,STEPPRICE,LASTSETTLEPRICE,PREVPRICE,IMTIME,BUYSELLFEE,SCALPERFEE,NEGOTIATEDFEE,EXERCISEFEE
284,SRH3,RFUD,SBRF-3.23,Фьючерсный контракт SBRF-3.23,13459.0,0,1.0,2023-03-16,2023-03-17,SR,...,15658.0,11598.0,1.0,13628.0,13459.0,2022-06-01 14:03:53,0.89,0.45,0.89,0.0
285,SRM2,RFUD,SBRF-6.22,Фьючерсный контракт SBRF-6.22,11901.0,0,1.0,2022-06-16,2022-06-17,SR,...,14091.0,10389.0,1.0,12240.0,11893.0,2022-06-01 14:03:53,0.78,0.39,0.78,0.0
286,SRM3,RFUD,SBRF-6.23,Фьючерсный контракт SBRF-6.23,13620.0,0,1.0,2023-06-15,2023-06-16,SR,...,15539.0,11467.0,1.0,13503.0,NaN,2022-06-01 14:03:53,0.90,0.45,0.90,0.0
287,SRU2,RFUD,SBRF-9.22,Фьючерсный контракт SBRF-9.22,12222.0,0,1.0,2022-09-15,2022-09-16,SR,...,14516.0,10616.0,1.0,12566.0,12201.0,2022-06-01 14:03:53,0.80,0.40,0.80,0.0
288,SRZ2,RFUD,SBRF-12.22,Фьючерсный контракт SBRF-12.22,12953.0,0,1.0,2022-12-15,2022-12-16,SR,...,15240.0,11190.0,1.0,13215.0,12977.0,2022-06-01 14:03:53,0.85,0.43,0.85,0.0


#### Получение данных минутных свечей по инструменту
Для получения данных минутных свечей по инструменту нам необходимо выполнить следующие шаги:
<ul>
    <li>получить boardid класса инструмента</li>
    <li>получить список торгуемых дат</li>
    <li>получить и сохранить данные по инструменту</li>
<ul>

#### Функция f_get_trading_dates
Возвращает торгуемый диапазон дат указанного инструмента
<br>Необходимый параметр: code - код инструмента

In [26]:
# функция для получения торгуемого диапазона дат инструмена
#---------------------------------------------------------------------------------------------------------------------------------
def f_get_trading_dates(code):
    #get boardid
    url_boardid='https://iss.moex.com/iss/engines/futures/markets/forts/boards.csv'
    boardid=pd.read_csv(url_boardid, sep=';',nrows=1,skiprows=1, encoding='windows-1251').boardid.iloc[0]
    #print('| code = '+code+' | boardid = '+boardid+' |')

    # запрос на торговый диапазон дат по инструменту
    url_candleborders='https://iss.moex.com/iss/engines/futures/markets/forts/boards/'+boardid+'/securities/'+code+'/candleborders.csv'
    print('get url: ' + url_candleborders)
    # запрашиваем данные по торговому диапазону дат инструмента и записываем в датафрейм
    df_candleborders=pd.read_csv(url_candleborders,sep=';',nrows=1,skiprows=1, encoding='windows-1251')
    #print(df_candleborders)

    # запрашиваем интервал и даты
    date_first=df_candleborders.begin.str[:10][0]
    date_last=df_candleborders.end.str[:10][0]
    interval=df_candleborders.interval[0]

    dictionary = {code: {'date_first': date_first, 'date_last': date_last, 'interval': interval}}
    #print('| code '+code+' |\n| interval '+str(interval)+' min |\n| date_first '+date_first+' |\n| date_last '+date_last+' |\n----------------------')
    return dictionary
#---------------------------------------------------------------------------------------------------------------------------------

In [27]:
# запросим торгуемый диапазон дат инструмента SRH9
code='SRH9'
f_get_trading_dates(code=code)

get url: https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/SRH9/candleborders.csv


{'SRH9': {'date_first': '2018-03-06',
  'date_last': '2019-03-21',
  'interval': 1}}

<b>Торгуемый диапазон дат кода инструмента SRH9:</b> <br>date_first: 2018-03-06 <br>date_last: 2019-03-21

<b>Получим минутные свечи interval=1 по коду инструмента code='SRH9' за 2019-03-21</b>

In [28]:

url_candles = 'https://iss.moex.com/iss/engines/futures/markets/forts/boards/'+boardid+'/'\
                'securities/'+code+'/candles.csv?interval=1&from=2019-03-21&till=2019-03-21&start=0'
df_url_candles = pd.read_csv(url_candles,sep=';',header=1)
print(df_url_candles.head())

    open  close   high    low  value  volume                begin  \
0  21057  21074  21098  21037      0     975  2019-03-21 10:00:00   
1  21065  21091  21098  21033      0    1812  2019-03-21 10:01:00   
2  21095  21090  21101  21079      0     862  2019-03-21 10:02:00   
3  21099  21096  21101  21084      0     935  2019-03-21 10:03:00   
4  21101  21102  21136  21092      0    1163  2019-03-21 10:04:00   

                   end  
0  2019-03-21 10:00:59  
1  2019-03-21 10:01:59  
2  2019-03-21 10:02:59  
3  2019-03-21 10:03:59  
4  2019-03-21 10:04:59  


Установим рабочую директорию для сохранения данных по инструменту

In [30]:
#if os.getcwd()!='d:\\Anaconda3\\My_scripts\\IIS_Moex\\historyIIS\\': os.chdir('d:\\Anaconda3\\My_scripts\\IIS_Moex\\historyIIS\\')
print(os.getcwd())

c:\Users\Administrator\repos\iis-api-moex\iis-api-moex


Запрсим минутные свечи по торговому инструменту за указанный диапазон дат и сохраним в каталог рабочей директории
<br>Для этого используем функцию <b>f_get_data_from_IIS_MOEX</b>
<br>Необходимые параметр: <b>code</b> - код инструмента, диапазон дат <b>date_first</b> и <b>date_last</b>

In [31]:
# функция запрашивает данные по торговому инструменту за указанный диапазон дат и сохраняет в каталог рабочей директории
#----------------------------------------------------------------------------------------------------------------------------------
def f_get_data_from_IIS_MOEX(code,date_first,date_last):
    #создаем лист всех дат из диапазона [date_first, date_last]
    date_list=[d.strftime('%Y-%m-%d') for d in pd.date_range(date_first,date_last)]
    #создаем папку для сохранения данных
    folder = code+'_'+ date_first +'_'+ date_last
    #folder=code+'_'+dictionary['SiH1']['date_end_prev']+'_'+dictionary['SiH1']['date_end']
    if not os.path.exists(folder): 
        os.makedirs(folder)
    print('| Folder to save data: ' + os.getcwd()+'\\' + folder + ' |')
    print('| Start extract data from server to |\n| '+code+' | from '+ date_first +' | till '+ date_last +' |\n'+45*'-')

    # получаем boardid
    url_boardid='https://iss.moex.com/iss/engines/futures/markets/forts/boards.csv'
    boardid=pd.read_csv(url_boardid, sep=';',nrows=1,skiprows=1, encoding='windows-1251').boardid.iloc[0]
    # цикл диапазона торгуемых дат по инструменту
    num_of_days=0
    #--------------------------------------------------------------------------------------------------------------------------------
    for date in date_list:
        print('| Get data from server: | '+code+' | '+date+' |')
        num_of_days=num_of_days + 1
        #----------------------------------------------------------------------------------------------------------------------------
        #максимальный объем данных запроса с сервера - 500 строк. Чтобы извлечь все данные необходимо увеличевать значение size в цикле
        for size in np.arange(0,501,500):
            
            #формируем url запрос и добавляем переменную start, т.е. с какой строки выполнить запрос занных с сервера 
            url_candles = 'https://iss.moex.com/iss/engines/futures/markets/forts/boards/'+boardid+'/'\
                'securities/'+code+'/candles.csv?interval=1&from='+date+'&till='+ date +'&start='+str(size)
            #делаем url запрос и сохраняем полученные данные в датафрейм
            df_url_candles = pd.read_csv(url_candles,sep=';',header=1)

            #если за указанную дату нет данных, то переходим к следующей дате
            if len(df_url_candles)==0 and size==0:
                #print('! no data for date | '+ code +' | '+ date +' | go to next date |')
                print('| ' +date + ' day off |')
                num_of_days=num_of_days - 1
                break
            
            try:
                len(df_hist.index)
            except:
                df_hist=df_url_candles.copy()
            
            if len(df_hist.index)==0:
                df_hist=df_url_candles.copy()
            elif size!=0:
                df_hist=df_hist.append(df_url_candles)
            
            pd.DataFrame.to_csv(df_hist, os.getcwd() +'\\'+ folder + '\\'+code+ '_'+date+'.txt',index=False,sep=';')
        #-------------------------------------------------------------------------------------------------------------------------
        #oчищаем данные
        try:
            df_hist=df_hist.iloc[0:0]
        except:
            pass
    #-----------------------------------------------------------------------------------------------------------------------------
    print('Finished extract data'+'\nNumber of trading days: '+str(num_of_days))
#---------------------------------------------------------------------------------------------------------------------------------

In [32]:
# вызовем функцию для получения и сохранения дат за указанный диапазон
f_get_data_from_IIS_MOEX( code=code
                         ,date_first='2019-03-10'
                         ,date_last='2019-03-21' )

| Folder to save data: c:\Users\Administrator\repos\iis-api-moex\iis-api-moex\SRH9_2019-03-10_2019-03-21 |
| Start extract data from server to |
| SRH9 | from 2019-03-10 | till 2019-03-21 |
---------------------------------------------
| Get data from server: | SRH9 | 2019-03-10 |
| 2019-03-10 day off |
| Get data from server: | SRH9 | 2019-03-11 |
| Get data from server: | SRH9 | 2019-03-12 |
| Get data from server: | SRH9 | 2019-03-13 |
| Get data from server: | SRH9 | 2019-03-14 |
| Get data from server: | SRH9 | 2019-03-15 |
| Get data from server: | SRH9 | 2019-03-16 |
| 2019-03-16 day off |
| Get data from server: | SRH9 | 2019-03-17 |
| 2019-03-17 day off |
| Get data from server: | SRH9 | 2019-03-18 |
| Get data from server: | SRH9 | 2019-03-19 |
| Get data from server: | SRH9 | 2019-03-20 |
| Get data from server: | SRH9 | 2019-03-21 |
Finished extract data
Number of trading days: 9


#### Заключение
Мы разобрали несоклько примеров url запросов csv и json к IIS серверу Московской бирже.
<br>Написали функции для извлечения и сохранения данных по интересующему коду инструмента за необходимый диапазон дат.